In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [4]:
data_preprocessed = pd.read_csv('Absenteeism_prepocessed.csv')

In [5]:
data_preprocessed.head()

,reason_1,reason_2,reason_3,reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [6]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [7]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours']>3,1,0)

In [8]:
data_preprocessed['Excessive Absenteeism'] = targets

In [9]:
targets.sum()/targets.shape[0]

0.45571428571428574

In [10]:
data_with_targets = data_preprocessed.drop('Absenteeism Time in Hours', axis = 1)

In [11]:
data_with_targets is data_preprocessed

False

In [12]:
data_with_targets.head()

,reason_1,reason_2,reason_3,reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


In [48]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

## scaling the dummies

In [14]:
from sklearn.preprocessing import StandardScaler

absenteeism_scaler = StandardScaler()
absenteeism_scaler.fit(unscaled_inputs)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
absenteeism_scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [17]:
scaled_inputs.shape

(700, 14)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_test,y_train,y_test =  train_test_split(scaled_inputs,targets, train_size = 0.8, random_state = 20)

In [20]:
print(x_train.shape,y_train.shape)

(560, 14) (560,)


In [21]:
print(x_test.shape,y_test.shape)

(140, 14) (140,)


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [23]:
reg = LogisticRegression()

In [24]:
reg.fit(x_train,y_train)

/home/ronit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
reg.score(x_train,y_train)

0.7839285714285714

In [26]:
model_outputs = reg.predict(x_train)

In [27]:
np.sum(model_outputs == y_train)/y_train.shape[0]

0.7839285714285714

In [28]:
reg.intercept_

array([-0.21957576])

In [29]:
reg.coef_

array([[ 2.07452213,  0.33466331,  1.56097327,  1.32750188,  0.1878226 ,
        -0.07060017,  0.70649077, -0.03995965, -0.2009517 , -0.00462411,
         0.31937669, -0.13558227,  0.38168134, -0.33338757]])

In [30]:
feature_name = unscaled_inputs.columns.values

In [31]:
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)

In [32]:
summary_table

,Feature name,Coefficient
0,reason_1,2.074522
1,reason_2,0.334663
2,reason_3,1.560973
3,reason_4,1.327502
4,Month Value,0.187823
5,Day of the week,-0.070600
6,Transportation Expense,0.706491
7,Distance to Work,-0.039960
8,Age,-0.200952
9,Daily Work Load Average,-0.004624


In [33]:
summary_table.index = summary_table.index +1

In [34]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()

In [35]:
summary_table['Oddes ratio']= np.exp(summary_table.Coefficient)

In [36]:
summary_table.sort_values('Oddes ratio', ascending='Fales')

,Feature name,Coefficient,Oddes ratio
14,Pets,-0.333388,0.716492
0,Intercept,-0.219576,0.802859
9,Age,-0.200952,0.817952
12,Education,-0.135582,0.873207
6,Day of the week,-0.070600,0.931834
8,Distance to Work,-0.039960,0.960828
10,Daily Work Load Average,-0.004624,0.995387
5,Month Value,0.187823,1.206619
11,Body Mass Index,0.319377,1.376270
2,reason_2,0.334663,1.397470


## scaling correctly without dummies

In [49]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [50]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy,with_mean, with_std)
        self.columns=columns
        self.mean_= None
        self.var_=None
        

    def fit(self,X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_=np.mean(X[self.columns])
        self.var_=np.var(X[self.columns])
        return self
    
    def transform(self,X,y=None,copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]),columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled],axis=1)[init_col_order]

In [51]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [57]:
colums_to_scale = ['Month Value',
       'Day of the week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index',
       'Children', 'Pets']


columns_to_omit = ['Day of the week','Daily Work Load Average','Distance to Work']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [64]:
columns_to_scale =unscaled_inputs.columns.values

In [69]:
absenteesim_scaler = CustomScaler(unscaled_inputs.columns.values)

In [70]:
absenteesim_scaler.fit(unscaled_inputs)

CustomScaler(columns=array(['reason_1', 'reason_2', 'reason_3', 'reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object),
             copy=None, with_mean=None, with_std=None)

In [71]:
scaled_inputs = absenteesim_scaler.transform(unscaled_inputs)

In [72]:
scaled_inputs

,reason_1,reason_2,reason_3,reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,-0.447980,0.880469,0.268487
1,-0.577350,-0.092981,-0.314485,-1.217485,0.182726,-1.574681,2.130803,1.002633,-0.447980,-0.019280,-0.589690
2,-0.577350,-0.092981,-0.314485,0.821365,0.182726,-0.654143,0.248310,1.002633,-0.447980,-0.919030,-0.589690
3,1.732051,-0.092981,-0.314485,-1.217485,0.182726,0.854936,0.405184,-0.643782,-0.447980,0.880469,-0.589690
4,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,-0.447980,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,-0.654143,0.562059,-1.114186,2.232242,0.880469,-0.589690
696,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,0.040034,-1.320435,-0.643782,-0.447980,-0.019280,1.126663
697,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,1.624567,-1.320435,-0.408580,2.232242,-0.919030,-0.589690
698,-0.577350,-0.092981,-0.314485,0.821365,-0.388293,0.190942,-0.692937,-0.408580,2.232242,-0.919030,-0.589690


## executing logistic regression again

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


x_train,x_test,y_train,y_test =  train_test_split(scaled_inputs,targets, train_size = 0.8, random_state = 20)
reg = LogisticRegression()
reg.fit(x_train,y_train)
reg_score = reg.score(x_train,y_train)
print("reg score is " ,reg_score)
model_outputs = reg.predict(x_train)
accuracy = np.sum(model_outputs == y_train)/y_train.shape[0]
print("accuracy is", accuracy)
feature_name = unscaled_inputs.columns.values
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index +1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table['Oddes ratio']= np.exp(summary_table.Coefficient)
summary_table = summary_table.sort_values('Oddes ratio', ascending= False)
print("summary_table \n", summary_table)



reg score is  0.7839285714285714
accuracy is 0.7839285714285714
summary_table 
               Feature name  Coefficient  Oddes ratio
1                 reason_1     2.067934     7.908464
3                 reason_3     1.561290     4.764966
4                 reason_4     1.310931     3.709627
6   Transportation Expense     0.691318     1.996345
10                Children     0.370100     1.447879
2                 reason_2     0.335434     1.398547
8          Body Mass Index     0.324361     1.383146
5              Month Value     0.184598     1.202735
9                Education    -0.126723     0.880977
7                      Age    -0.198284     0.820137
0                Intercept    -0.221213     0.801546
11                    Pets    -0.325471     0.722187


/home/ronit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## dropping unmeaningfull featuers

In [ ]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the week',
                                           'Daily Work Load Average',
                                           'Distance to Work'], axis = 1)

In [ ]:
unscaled_inputs = data_with_targets.iloc[:,:-1]
columns_to_omit = ['Day of the week','Daily Work Load Average','Distance to Work']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

absenteesim_scaler = CustomScaler(colums_to_scale)
absenteesim_scaler.fit(unscaled_inputs)
scaled_inputs = absenteesim_scaler.transform(unscaled_inputs)

x_train,x_test,y_train,y_test =  train_test_split(scaled_inputs,targets, train_size = 0.8, random_state = 20)
reg = LogisticRegression()
reg.fit(x_train,y_train)
reg_score = reg.score(x_train,y_train)
print("reg score is " ,reg_score)
model_outputs = reg.predict(x_train)
accuracy = np.sum(model_outputs == y_train)/y_train.shape[0]
print("accuracy is", accuracy)
feature_name = unscaled_inputs.columns.values
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index +1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table['Oddes ratio']= np.exp(summary_table.Coefficient)
summary_table = summary_table.sort_values('Oddes ratio', ascending= False)
print("summary_table \n", summary_table)


## test the model

In [75]:
reg.predict(x_test).round(2)

array([0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0])

In [ ]:
reg.score(x_test,y_test)

In [ ]:
predict_proba = reg.predict_proba(x_test)
predict_proba

In [ ]:
predict_proba.shape

In [ ]:
predict_proba[:,1]

## saving the model

In [ ]:
import pickle

In [ ]:
with open('model','wb') as file:
    pickle.dump(reg,file)

In [ ]:
with open('scaler','wb') as file:
    pickle.dump(absenteesim_scaler,file)